In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain.agents.openai_functions import (
    create_openai_functions_agent,
    OpenAIFunctionsAgentPrompt
)


ModuleNotFoundError: No module named 'langchain.agents.openai_functions'

In [3]:
# agentic_rag_multitool.py

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_openai_functions_agent
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import tool
from typing import TypedDict, Annotated, List, Optional
import math


# ---- STATE ----
class AgentState(TypedDict):
    question: str
    answer: Optional[str]
    messages: List


# ---- TOOLS ----
@tool
def retrieve_docs(query: str) -> str:
    """Search relevant documents in the knowledge base."""
    print("📚 [Tool] retrieve_docs()")
    vectorstore = FAISS.load_local("vectorstore", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
    docs = vectorstore.similarity_search(query, k=3)
    return "\n".join([d.page_content for d in docs])


@tool
def math_tool(expression: str) -> str:
    """Solve a math expression like '2+2*5'."""
    print("🧮 [Tool] math_tool()")
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Math error: {str(e)}"


@tool
def python_tool(code: str) -> str:
    """Run simple Python code and return the result."""
    print("🐍 [Tool] python_tool()")
    try:
        local_vars = {}
        exec(code, {}, local_vars)
        return str(local_vars)
    except Exception as e:
        return f"Python error: {str(e)}"


@tool
def web_search(query: str) -> str:
    """Simulate a web search (replace with real API)."""
    print("🌐 [Tool] web_search()")
    return f"Simulated web result for '{query}'."


# ---- TOOL LIST ----
tools = [
    Tool(name="retrieve_docs", func=retrieve_docs, description="Search internal knowledge base"),
    Tool(name="math_tool", func=math_tool, description="Evaluate math expressions"),
    Tool(name="python_tool", func=python_tool, description="Execute simple Python code"),
    Tool(name="web_search", func=web_search, description="Search the web for current info"),
]


# ---- AGENT SETUP ----
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# ---- GRAPH NODE ----
def agent_node(state: AgentState):
    question = state["question"]
    result = agent_executor.invoke({"input": question})
    return {
        "question": question,
        "answer": result["output"],
        "messages": [result],
    }


# ---- LANGGRAPH ----
builder = StateGraph(AgentState)
builder.add_node("agent_node", agent_node)
builder.set_entry_point("agent_node")
builder.set_finish_point("agent_node")
graph = builder.compile()


# ---- RUN ----
if __name__ == "__main__":
    print("🤖 Welcome to the Agentic RAG Multitool Chatbot!")

    print("\n💬 Ask a question (Ctrl+C to exit)\n")
    while True:
        try:
            question = input("🧑‍💻 You: ")
            output = graph.invoke({"question": question})
            print("🤖 AI:", output["answer"], "\n")
        except KeyboardInterrupt:
            print("\n👋 Exiting.")
            break


C:\Users\sanat\AppData\Local\Temp\ipykernel_29076\2629238541.py:70: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


TypeError: create_openai_functions_agent() missing 1 required positional argument: 'prompt'